# OLD APPROACH

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleYOLO  # clone
!cd PaddleYOLO
!pip install -r requirements.txt

In [ ]:
# 1.训练（单卡/多卡），加 --eval 表示边训边评估，加 --amp 表示混合精度训练
!cd PaddleYOLO && CUDA_VISIBLE_DEVICES=0 python tools/train.py -c PaddleDetection/PaddleYOLO/configs/custom/yolov8_m_500e_coco.yml --eval --amp

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python tools/export_model.py -c configs/mot/bytetrack/detector/ppyoloe_crn_l_36e_640x640_mot17half.yml -o weights=https://paddledet.bj.bcebos.com/models/mot/ppyoloe_crn_l_36e_640x640_mot17half.pdparams
%cd PaddleDetection
!CUDA_VISIBLE_DEVICES=0 python tools/export_model.py -c /home/5qx9nf8a/team_workspace/PaddleDetection/configs/mot/bytetrack/detector/ppyoloe_plus_l_bytetrack.yml -o weights=/home/5qx9nf8a/team_workspace/PaddleDetection//tracking/model_final.pdparams
    

In [ ]:
model.info()

In [ ]:
model.val()

In [ ]:
from tracking.tracking_counter import create_submission_dict, write_submission

#submission_dict = create_submission_dict(scanned_racks=scanner.rack_tracks, mAP=96.3, fps=19.47)99.71
submission_dict = create_submission_dict(scanned_racks=saved_racks, mAP=96.3, fps=0) 

write_submission(submission_dict=submission_dict, submission_path="temp/AcademicWeapons.json")


In [ ]:
video = "data/Hackathon_Stage2/Evaluation_set/video/eval_video_1.mp4"
img = "data/Hackathon_Stage2/Evaluation_set/dataset/images/109.jpg"
engine = "runs/detect/train2/weights/best.engine"
data = "data/trans_data/val/images"
import cv2
im = cv2.imread(img)[None]
print(im.shape)
from ultralytics.yolo.v8.detect.predict import DetectionPredictor
predictor = DetectionPredictor() #CustomPredictor()#
predictor.setup_model(model=engine)
predictor.imgsz = (3, 960, 960)
predictor.args.mode = "predict"
predictor.args.imgsz = 960
predictor.args.save = False
predictor.args.cache = True



In [ ]:
from time import time

with VideoSink(TARGET_VIDEO_PATH, 1, video_info) as sink:
    detect_time, track_up_time, track_matcher, box_anno, scanner_up, scanner_anno, writer = 0,0,0,0,0,0,0
    
    for idx, frame in tqdm(enumerate(generator), total=video_info.total_frames):
        
        start = time()
        results = predictor(source=frame)
        end = time()
        detect_time += end - start
        
        detections = Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int)
        )
        if detections.xyxy.shape[0] != 0:
            # tracking detections
            start = time()
            tracks = byte_tracker.update(
                output_results=detections2boxes(detections=detections),
                img_info=frame.shape,
                img_size=frame.shape
            )
            end = time()
            track_up_time += end - start
            
            start = time()
            tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
            end = time()
            track_matcher += end - start
            
            detections.tracker_id = np.array(tracker_id)

            # filtering out detections without trackers
            mask = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
            detections.filter(mask=mask, inplace=True)
            detection_list.append(detections)
            
            # format custom labels
            labels = [
                f"#{tracker_id} {CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
                for _, confidence, class_id, tracker_id
                in detections
            ]
            
            # annotatoe detection boxes
            start = time()
            #frame = box_annotator.annotate(
            #    frame=frame, detections=detections, labels=labels
            #)
            end = time()
            box_anno += end - start
            
            # update the scanner
            start = time()
            scanner.update(detections=detections)
            end = time()
            scanner_up += end - start
            # draw the scanner
            start = time()
            #scanner_annotator.annotate(frame=frame, rack_scanner=scanner)
            end = time()
            scanner_anno += end - start
            # add the annotated frame to video
        start = time()
        sink.write_frame(frame)
        end = time()
        writer += end - start
print(round(detect_time, 3)*  1000)
print(round(track_up_time, 3)* 1000)
print(round(track_matcher, 3)* 1000)
print(round(box_anno, 3)* 1000)
print(round(scanner_up, 3)* 1000)
print(round(scanner_anno, 3)* 1000)
print(round(writer, 3)* 1000)